In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')

In [3]:
# Age 범주형 변경

# 구간 설정
bins = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, float('inf')]

# 각 구간에 대한 라벨 설정
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

# Age 열을 범주형 변수로 변환
train['AgeGroup'] = pd.cut(train['Age'], bins=bins, labels=labels, right=False)

In [4]:
# Age 열 삭제
train = train.drop(['Age'], axis = 1)

In [5]:
train['AgeGroup'] = train['AgeGroup'].astype(int)

In [6]:
train.shape

(11504798, 12)

In [7]:
train.head()

,id,Gender,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,AgeGroup
0,0,Male,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0,0
1,1,Male,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1,4
2,2,Female,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0,1
3,3,Female,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0,3
4,4,Female,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0,3


In [8]:
# IQR을 이용하여 이상치 제거
Q1 = train['Annual_Premium'].quantile(0.25)
Q3 = train['Annual_Premium'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치가 있는 행 제거
deleted_train = train[(train['Annual_Premium'] >= lower_bound) & (train['Annual_Premium'] <= upper_bound)]

deleted_train.shape

(9127525, 12)

In [9]:
from sklearn.preprocessing import MinMaxScaler

numerical = ['Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage','AgeGroup']
scaler = MinMaxScaler()
deleted_train[numerical] = scaler.fit_transform(deleted_train[numerical])
deleted_train.head()


/tmp/ipykernel_283/3131530773.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deleted_train[numerical] = scaler.fit_transform(deleted_train[numerical])


,id,Gender,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,AgeGroup
1,1,Male,1,0.538462,0,> 2 Years,Yes,0.968248,0.154321,0.961938,1,0.333333
2,2,Female,1,0.269231,1,< 1 Year,No,0.600141,0.932099,0.844291,0,0.083333
4,4,Female,1,0.288462,1,1-2 Year,No,0.492679,0.932099,0.982699,0,0.250000
5,5,Female,1,0.903846,1,< 1 Year,No,0.425631,0.932099,0.647059,0,0.166667
6,6,Male,1,0.865385,1,< 1 Year,No,0.407603,0.932099,0.622837,0,0.000000


In [10]:
deleted_train.head()

,id,Gender,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,AgeGroup
1,1,Male,1,0.538462,0,> 2 Years,Yes,0.968248,0.154321,0.961938,1,0.333333
2,2,Female,1,0.269231,1,< 1 Year,No,0.600141,0.932099,0.844291,0,0.083333
4,4,Female,1,0.288462,1,1-2 Year,No,0.492679,0.932099,0.982699,0,0.250000
5,5,Female,1,0.903846,1,< 1 Year,No,0.425631,0.932099,0.647059,0,0.166667
6,6,Male,1,0.865385,1,< 1 Year,No,0.407603,0.932099,0.622837,0,0.000000


In [11]:
deleted_train.drop(['id'], axis = 1)

,Gender,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,AgeGroup
1,Male,1,0.538462,0,> 2 Years,Yes,0.968248,0.154321,0.961938,1,0.333333
2,Female,1,0.269231,1,< 1 Year,No,0.600141,0.932099,0.844291,0,0.083333
4,Female,1,0.288462,1,1-2 Year,No,0.492679,0.932099,0.982699,0,0.250000
5,Female,1,0.903846,1,< 1 Year,No,0.425631,0.932099,0.647059,0,0.166667
6,Male,1,0.865385,1,< 1 Year,No,0.407603,0.932099,0.622837,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
11504792,Male,1,0.538462,1,1-2 Year,No,0.457806,0.944444,0.660900,0,0.166667
11504793,Male,1,0.115385,0,1-2 Year,Yes,0.412612,0.154321,0.719723,0,0.416667
11504794,Female,1,0.692308,0,< 1 Year,Yes,0.449603,0.932099,0.363322,1,0.083333
11504796,Female,1,0.538462,0,1-2 Year,Yes,0.783595,0.154321,0.913495,1,0.500000


In [12]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cat=['Gender', 'Vehicle_Age', 'Vehicle_Damage']
for column in cat:
    deleted_train[column] = label_encoder.fit_transform(deleted_train[column])

/tmp/ipykernel_283/2964673091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deleted_train[column] = label_encoder.fit_transform(deleted_train[column])
/tmp/ipykernel_283/2964673091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deleted_train[column] = label_encoder.fit_transform(deleted_train[column])
/tmp/ipykernel_283/2964673091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [13]:
deleted_train.head()

,id,Gender,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,AgeGroup
1,1,1,1,0.538462,0,2,1,0.968248,0.154321,0.961938,1,0.333333
2,2,0,1,0.269231,1,1,0,0.600141,0.932099,0.844291,0,0.083333
4,4,0,1,0.288462,1,0,0,0.492679,0.932099,0.982699,0,0.250000
5,5,0,1,0.903846,1,1,0,0.425631,0.932099,0.647059,0,0.166667
6,6,1,1,0.865385,1,1,0,0.407603,0.932099,0.622837,0,0.000000


In [14]:
deleted_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9127525 entries, 1 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                int64  
 2   Driving_License       int64  
 3   Region_Code           float64
 4   Previously_Insured    int64  
 5   Vehicle_Age           int64  
 6   Vehicle_Damage        int64  
 7   Annual_Premium        float64
 8   Policy_Sales_Channel  float64
 9   Vintage               float64
 10  Response              int64  
 11  AgeGroup              float64
dtypes: float64(5), int64(7)
memory usage: 905.3 MB
